# 1 Workflow planning with new


In [3]:
! pip install google-generativeai
! pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json

In [ ]:
questions = [
    "1. Mục tiêu chính của bạn là gì (Học tập / Làm việc / Sức khỏe / Phát triển cá nhân / Khác...)?",
    "2. Bạn rảnh lúc nào trong ngày? (Sáng / Trưa / Tối / Cuối tuần...)?",
    "3. Bạn muốn hệ thống lên kế hoạch theo: (Ngày / Tuần)?",
    "4. Ưu tiên của bạn là gì? (Ví dụ: Sức khỏe > Học tập > Công việc)",
    "5. Thói quen ngủ / giờ thức dậy?",
]

answers = {}

for i, question in enumerate(questions, start=1):
    user_input = input(f"Bot: {question}\nBạn: ")
    if user_input.lower() in ["thoát", "exit", "bye", "tạm biệt"]:
        print("Bot: Tạm biệt nhé!")
        break
    answers[f"{i}"] = user_input

# (1) Time managerment AI Agent


In [ ]:
load_dotenv()
AI_KEY = os.getenv("AI_API_KEY")
genai.configure(api_key=AI_KEY)

# Khởi tạo model
model_gemini = genai.GenerativeModel("gemini-2.0-flash")

answers = {
    "goal": ["Học tập", "Làm việc", "Sức khỏe"],
    "availableTime": "Cuối tuần",
    "planningStyle": "Tuần",
    "priority": ["Sức khỏe", "Học tập", "Công việc"],
    "wakeUp": "06:30",
    "sleep": "00:00",
}

# Prompt
prompt = f"""
Bạn là một trợ lý lập kế hoạch AI thông minh.
Bạn sẽ giúp người dùng lên kế hoạch cho tuần đầu tiên của họ dựa trên thông tin họ cung cấp.

Dữ liệu người dùng:
{json.dumps(answers, ensure_ascii=False)}

🔹 Từ Thứ Hai đến Thứ Sáu:
- Tập trung vào năng suất cao: học tập, làm việc, thể chất.
- Có thời gian nghỉ xen kẽ để tránh quá tải.
- Bắt đầu sau giờ thức dậy và kết thúc trước giờ ngủ.

🔹 Thứ Bảy và Chủ Nhật:
- Lịch nhẹ nhàng, tập trung thư giãn, nghỉ ngơi.
- Vẫn có thể duy trì tập luyện nhẹ và tổng kết tuần.

🔹 Lưu ý:
- Lên kế hoạch tuần dựa trên thứ tự ưu tiên mục tiêu người dùng cung cấp.
- Tất cả thời gian hoạt động cần nằm trong khung từ giờ thức dậy đến giờ ngủ.
- Mỗi ngày nên có ít nhất 1–2 khung giờ nghỉ hoặc thư giãn.

📦 Yêu cầu:
- Trả về kết quả ở định dạng JSON, đúng cấu trúc sau:

{{
  "weekPlan": {{
    "monday": [
      {{ "time": "06:30-07:15", "activity": "Tập thể dục buổi sáng", "note": "Cardio nhẹ tại nhà" }},
      ...
    ],
    ...
    "sunday": [ ... ]
  }}
}}

Trả về **chỉ nội dung JSON thuần**, không thêm mô tả, không sử dụng markdown (```), không thêm đoạn văn bản nào khác.
"""

response = model_gemini.generate_content(prompt)

response_text = response.text.strip()

if response_text.startswith("```json"):
    response_text = response_text.replace("```json", "").replace("```", "").strip()

try:
    plan = json.loads(response_text)
    print("✅ Kế hoạch AI đã lên:")
    print(json.dumps(plan, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("❌ Không parse được JSON:", e)
    print("Raw response:\n", response_text)

✅ Kế hoạch AI đã lên:
{
  "weekPlan": {
    "monday": [
      {
        "time": "06:30-07:00",
        "activity": "Vệ sinh cá nhân",
        "note": ""
      },
      {
        "time": "07:00-07:30",
        "activity": "Ăn sáng",
        "note": ""
      },
      {
        "time": "07:30-09:30",
        "activity": "Học tập",
        "note": "Ôn tập kiến thức cũ"
      },
      {
        "time": "09:30-10:00",
        "activity": "Nghỉ ngơi",
        "note": "Uống trà, nghe nhạc"
      },
      {
        "time": "10:00-12:00",
        "activity": "Làm việc",
        "note": "Giải quyết công việc tồn đọng"
      },
      {
        "time": "12:00-13:00",
        "activity": "Ăn trưa",
        "note": ""
      },
      {
        "time": "13:00-15:00",
        "activity": "Học tập",
        "note": "Đọc tài liệu chuyên ngành"
      },
      {
        "time": "15:00-15:30",
        "activity": "Nghỉ ngơi",
        "note": "Đi bộ nhẹ nhàng"
      },
      {
        "time": "15:30-17:30",
 

# (2) Agent tư vấn


In [ ]:
# Prompt
prompt = f"""
Bạn là một trợ lý lập kế hoạch AI thông minh.
Bạn sẽ tư vấn giúp người dùng những thói quen tốt và thói quen xấu để họ có thể cải thiện sức khỏe và năng suất làm việc của mình.
Chỉ tập trung dựa vào những hoạt động của người dùng thui như ngủ nghỉ hay ăn uống 
Dữ liệu người dùng:
{json.dumps(answers, ensure_ascii=False)}
📦 Yêu cầu:
- Trả về kết quả ở định dạng JSON, đúng cấu trúc sau:
    
    {{
    "goodHabits": [
        "Thói quen tốt 1",
        "Thói quen tốt 2",
        ...
    ],
    "badHabits": [
        "Thói quen xấu 1",
        "Thói quen xấu 2",
        ...
    ]
    }}
"""
response = model_gemini.generate_content(prompt)
response_text = response.text.strip()
if response_text.startswith("```json"):
    response_text = response_text.replace("```json", "").replace("```", "").strip()
    print(response_text)
try:
    habits = json.loads(response_text)
    print("✅ Thói quen AI đã tư vấn:")
    print(json.dumps(habits, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("❌ Không parse được JSON:", e)
    print("Raw response:\n", response_text)

{
  "goodHabits": [
    "Dậy sớm lúc 6:30 giúp bạn có thêm thời gian cho các hoạt động buổi sáng như tập thể dục, ăn sáng đầy đủ và chuẩn bị cho ngày mới.",
    "Ưu tiên sức khỏe hàng đầu là một thói quen tốt, đảm bảo bạn có đủ năng lượng và tinh thần để học tập và làm việc hiệu quả.",
    "Dành thời gian cuối tuần để lên kế hoạch cho tuần tiếp theo giúp bạn có một lộ trình rõ ràng và giảm căng thẳng.",
    "Duy trì thời gian biểu ngủ nghỉ đều đặn, kể cả vào cuối tuần, để ổn định đồng hồ sinh học."
  ],
  "badHabits": [
    "Ngủ lúc 0:00 có thể dẫn đến thiếu ngủ, ảnh hưởng tiêu cực đến sức khỏe, khả năng tập trung và hiệu suất làm việc.",
    "Không có thời gian biểu cụ thể cho các bữa ăn có thể dẫn đến ăn uống không lành mạnh và ảnh hưởng đến sức khỏe tổng thể.",
    "Bỏ qua việc vận động thể chất có thể gây ra các vấn đề sức khỏe về lâu dài."
  ]
}
✅ Thói quen AI đã tư vấn:
{
  "goodHabits": [
    "Dậy sớm lúc 6:30 giúp bạn có thêm thời gian cho các hoạt động buổi sáng như tập thể dụ

# (3) Adaptive AI Agent (Edit short time)


In [16]:
reason = "Hiện giờ là 15PM, tôi cảm thấy rất mệt mỏi và áp lực với công việc đến lỗi suy nghĩ quẩn tôi nên làm gì để giải tỏa đây."
prompt = f"""
Bạn là một trợ lý lập kế hoạch AI thông minh, biết lắng nghe và thấu hiểu cảm xúc của con người.

Thông tin người dùng cung cấp:
- Lý do hiện tại: {reason}

Nhiệm vụ của bạn:
- Phân tích lý do người dùng cung cấp để xác định:
    - Họ đang mệt mỏi cần nghỉ ngơi? Phân tích cảm xúc của người dùng từ câu lý do trên (ví dụ: mệt mỏi, áp lực, buồn bã, lo lắng, mất động lực...).
    - Hay họ có lịch trình đột xuất (ví dụ: gặp bạn bè, đi công việc...)?
- Dựa trên phân tích:
    - Nếu mệt mỏi → giảm khối lượng công việc, thay bằng hoạt động nhẹ nhàng, nghỉ ngơi.
        + Tạo ra một đoạn "Advise" dựa trên cảm xúc phân tích được:
        + Viết thành **một đoạn văn ngắn (4-6 câu)**.
        + Ngôn từ chân thành, sâu sắc, như một người bạn thực sự tâm sự, khích lệ.
        + Gợi ý người dùng nghỉ ngơi, thư giãn hoặc nhẹ nhàng lấy lại năng lượng.
    - Nếu bận đột xuất → hủy, điều chỉnh, hoặc ghi chú lại các hoạt động trùng giờ đó.
        + Không cần tạo ra một đoạn "Advise" như trên.
        + Bảo lịch mình đã điều chỉnh
- Xác định thời điểm người dùng để làm mốc điều chỉnh lịch trình.
- **Chỉ điều chỉnh các hoạt động từ sau giờ đó trở đi**, không cần giữ lịch trình trước đó. Nếu user đưa ra lịch thì ghi lại từ giờ đó trở đi
- Trả về kết quả **chỉ dưới dạng JSON**, đúng mẫu sau:
  {{
    "Advise"or "Reply": "Đoạn lời khuyên sâu sắc cá nhân hóa theo cảm xúc người dùng."or"Mình đã điều chỉnh lịch trình của bạn như sau",
    "monday": [
      {{
        "time": "15:30-16:00",
        "activity": "Đi bộ nhẹ nhàng ngoài trời",
        "note": "Giúp thư giãn tinh thần và hồi phục năng lượng"
      }},
      ...
    ]
  }}
- Tuyệt đối **không** thêm bất kỳ giải thích nào bên ngoài JSON.

Dữ liệu lịch trình gốc ngày hôm nay:
{json.dumps(plan.get("weekPlan").get("monday"), ensure_ascii=False)}
"""


response = model_gemini.generate_content(prompt)
response_text = response.text.strip()

if response_text.startswith("```json"):
    response_text = response_text.replace("```json", "").replace("```", "").strip()

try:
    planningadapter = json.loads(response_text)
    print("✅ Kế hoạch AI đã lên:")
    print(json.dumps(planningadapter, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("❌ Không parse được JSON:", e)
    print("Raw response:\n", response_text)

✅ Kế hoạch AI đã lên:
{
  "Advise": "Mình hiểu rằng bạn đang cảm thấy rất mệt mỏi và áp lực. Đừng cố gắng gồng mình nữa, bạn nhé. Hãy cho phép bản thân được chậm lại một chút. Mình nghĩ bạn nên dành thời gian này để thực sự nghỉ ngơi, làm điều gì đó mình thích, hoặc đơn giản chỉ là ngồi yên tĩnh và hít thở sâu. Bạn xứng đáng được yêu thương và chăm sóc, cả về thể chất lẫn tinh thần.",
  "monday": [
    {
      "time": "15:30-16:00",
      "activity": "Đi bộ nhẹ nhàng ngoài trời",
      "note": "Giúp thư giãn tinh thần và hồi phục năng lượng"
    },
    {
      "time": "16:00-17:30",
      "activity": "Nghe nhạc hoặc đọc sách",
      "note": "Chọn một không gian yên tĩnh và thư giãn"
    },
    {
      "time": "17:30-18:30",
      "activity": "Tập thể dục nhẹ nhàng",
      "note": "Yoga hoặc đi bộ nhẹ nhàng"
    },
    {
      "time": "18:30-19:30",
      "activity": "Ăn tối",
      "note": ""
    },
    {
      "time": "19:30-22:00",
      "activity": "Xem phim hoặc trò chuyện với bạn 

# (4) Update Schedular (long Time)


In [27]:
reason = "Tôi có một cuộc họp quan trọng từ 15:00 đến 19:00 vào thứ 2, thứ 3 và thứ 5 tuần này."
prompt = f"""
Bạn là một trợ lý AI chuyên lập kế hoạch thông minh.

Thông tin người dùng:
- {reason}

Dữ liệu lịch trình gốc tuần này:
{json.dumps(plan.get("weekPlan"), ensure_ascii=False)}

Yêu cầu:
1. Phân tích kỹ nội dung 'reason' để xác định:
   - Các NGÀY cần điều chỉnh.
   - Các KHOẢNG THỜI GIAN cụ thể cần xử lý (ví dụ: 17:00-20:00).

2. Đối với mỗi ngày cần điều chỉnh:
   - Duyệt qua toàn bộ các hoạt động trong lịch gốc:
     - Nếu hoạt động:
       - **Bắt đầu trước** và **kết thúc trong khoảng** => Cắt phần giao với khoảng yêu cầu.
       - **Bắt đầu trong khoảng** và **kết thúc sau khoảng** => Cắt phần giao với khoảng yêu cầu.
       - **Trùng hoàn toàn trong khoảng** => Giữ nguyên hoạt động đó.
       - **Nằm ngoài khoảng thời gian** => Bỏ qua (không đưa vào lịch mới).
   - Sau khi cắt, nếu không có hoạt động nào trong khoảng thời gian yêu cầu, hãy **tạo mới** một hoạt động mới đúng nội dung 'reason'.
   - Vẫn đảm thời gian nghỉ ngơi, ăn uống và làm việc phù hợp

3. Quy tắc bổ sung:
   - Nội dung hoạt động mới nên lấy từ 'reason'.
   - Note mặc định cho hoạt động thêm mới là: "Theo yêu cầu người dùng".


4. Định dạng kết quả:
   - Trả về **chỉ JSON** đúng mẫu sau:
{{
  "weekPlan": {{
    "monday": [
      {{ "time": "17:00-20:00", "activity": "Học tiếng Anh", "note": "Theo yêu cầu người dùng" }}
    ],
    ...
  }}
}}
   - Không giải thích, không thêm văn bản ngoài JSON.
   - Các ngày không liên quan không cần xuất hiện trong kết quả.

Ghi chú thêm:
- Khi cắt hoạt động, đảm bảo thời gian hợp lệ (giờ bắt đầu < giờ kết thúc).
- Nếu hoạt động bị chia ra, mỗi phần mới vẫn giữ nguyên 'activity' và 'note' ban đầu.

Hãy bắt đầu thực hiện chính xác theo yêu cầu trên.
"""


response = model_gemini.generate_content(prompt)
response_text = response.text.strip()

if response_text.startswith("```json"):
    response_text = response_text.replace("```json", "").replace("```", "").strip()

try:
    planningUpdate = json.loads(response_text)
    print("✅ Kế hoạch AI đã lên:")
    print(json.dumps(planningUpdate, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("❌ Không parse được JSON:", e)
    print("Raw response:\n", response_text)

✅ Kế hoạch AI đã lên:
{
  "weekPlan": {
    "monday": [
      {
        "time": "06:30-07:00",
        "activity": "Vệ sinh cá nhân",
        "note": ""
      },
      {
        "time": "07:00-07:30",
        "activity": "Ăn sáng",
        "note": ""
      },
      {
        "time": "07:30-09:30",
        "activity": "Học tập",
        "note": "Ôn tập kiến thức cũ"
      },
      {
        "time": "09:30-10:00",
        "activity": "Nghỉ ngơi",
        "note": "Uống trà, nghe nhạc"
      },
      {
        "time": "10:00-12:00",
        "activity": "Làm việc",
        "note": "Giải quyết công việc tồn đọng"
      },
      {
        "time": "12:00-13:00",
        "activity": "Ăn trưa",
        "note": ""
      },
      {
        "time": "13:00-15:00",
        "activity": "Học tập",
        "note": "Đọc tài liệu chuyên ngành"
      },
      {
        "time": "15:00-19:00",
        "activity": "Cuộc họp quan trọng",
        "note": "Theo yêu cầu người dùng"
      },
      {
        "time"

# (5) Food advice AI Agent


In [ ]:
reason = "Tôi có cá ngừ, rau cải cà chua và trứng trong tủ lạnh và trưa tui muốn ăn cơm . Tôi muốn nấu một món ăn từ các nguyên liệu này."
prompt = f"""
Bạn là một trợ lý AI chuyên lập kế hoạch ăn uống thông minh.

Thông tin người dùng:
- Nguyên liệu hiện có: {reason}

Dữ liệu lịch trình gốc trong ngày:
{json.dumps(plan.get("monday"), ensure_ascii=False)}

Yêu cầu:
1. Phân tích danh sách nguyên liệu ('reason') để:
   - Gợi ý món ăn phù hợp cho từng bữa ăn trong ngày:
   - Nếu các giờ ăn uống đã qua thì bỏ qua lên kế hoạch bữa đó.
   - Nếu còn lại các bữa ăn chưa qua thì:
     + Ăn sáng: món đơn giản, dễ chế biến nhanh.
     + Ăn trưa, Ăn tối: món chính, có thể cầu kỳ hơn.
   - Các bữa ăn phải chọn món **khác nhau**, không được lặp lại món giữa các buổi.

2. Nếu nguyên liệu người dùng không đủ để chế biến một món ăn hợp lý:
   - Tự động **bổ sung nguyên liệu cần thiết** vào 'note' kèm theo ghi chú: "(cần thêm)".
   - Không được bỏ qua bữa ăn.

3. Duyệt qua toàn bộ hoạt động trong ngày:
   - Nếu hoạt động liên quan đến ăn uống ("Ăn sáng", "Ăn trưa", "Ăn tối"):
     - Cập nhật:
       - 'activity': thay bằng tên món ăn gợi ý.
       - 'note': liệt kê đầy đủ nguyên liệu đã sử dụng, đánh dấu rõ những nguyên liệu cần bổ sung.
     - Nếu món ăn cần thời gian nấu lâu hơn khoảng thời gian hiện tại:
       - Tinh chỉnh lại khoảng thời gian hoạt động **trong cùng ngày**, đảm bảo:
         + Bắt đầu sớm hơn nếu cần.
         + Hoặc kết thúc trễ hơn một chút, nhưng không vượt quá các mốc thời gian sinh hoạt quan trọng khác (ví dụ: không đẩy trễ quá giờ ngủ).

4. Các hoạt động khác (không phải ăn uống) giữ nguyên, không thay đổi.

5. Nếu trong lịch trình gốc **không có hoạt động ăn uống**:
   - Tự động **thêm mới** hoạt động "Nấu ăn" và "Ăn uống" với khung giờ hợp lý trong ngày.

6. Định dạng kết quả:
   - Trả về **chỉ JSON** đúng mẫu:
[
  {{ "time": "06:30-07:15", "activity": "Bánh mì trứng ốp la", "note": "Nguyên liệu: bánh mì, trứng, bơ (cần thêm)...." }},
  {{ "time": "12:00-13:30", "activity": "Cơm gà luộc", "note": "Nguyên liệu: gạo, gà, hành lá" }},
  {{ "time": "19:00-20:00", "activity": "Canh bí đỏ nấu tôm", "note": "Nguyên liệu: bí đỏ, tôm, hành tím (cần thêm)...." }},
  ...
]

- Mỗi phần tử là một hoạt động trong ngày.
- Tuyệt đối không được giải thích thêm gì ngoài JSON.

Ghi chú:
- Nếu tinh chỉnh giờ, đảm bảo start time < end time và hợp lý với nhịp sinh hoạt.
- Các món ăn phải phù hợp với bữa ăn và nguyên liệu.
- Không lặp lại món giữa các buổi.

Hãy bắt đầu xử lý chính xác theo yêu cầu trên.
"""


response = model_gemini.generate_content(prompt)
response_text = response.text.strip()

if response_text.startswith("```json"):
    response_text = response_text.replace("```json", "").replace("```", "").strip()

try:
    planningUpdate = json.loads(response_text)
    print("✅ Kế hoạch AI đã lên:")
    print(json.dumps(planningUpdate, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("❌ Không parse được JSON:", e)
    print("Raw response:\n", response_text)

✅ Kế hoạch AI đã lên:
[
  {
    "time": "07:00-07:30",
    "activity": "Trứng chiên cà chua",
    "note": "Nguyên liệu: trứng, cà chua, hành lá (cần thêm), gia vị"
  },
  {
    "time": "12:00-13:00",
    "activity": "Cơm cá ngừ sốt cà chua",
    "note": "Nguyên liệu: cá ngừ, cà chua, hành tây (cần thêm), gia vị, cơm"
  },
  {
    "time": "19:00-19:30",
    "activity": "Salad rau cải trộn trứng",
    "note": "Nguyên liệu: rau cải, trứng luộc, sốt mè rang (cần thêm)"
  }
]
